# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
path = "../output_data/city_weather.csv"
cities = pd.read_csv(path)
cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Grand Forks,47.94,-97.43,271.15,68,40,3.60,US,1603000422
1,1,Mataura,-46.19,168.86,283.71,68,85,2.68,NZ,1603000422
2,2,Ushuaia,-54.80,-68.30,278.15,86,75,2.60,AR,1603000422
3,3,Ushuaia,-54.80,-68.30,278.15,86,75,2.60,AR,1603000422
4,4,Revda,56.80,59.93,275.15,69,75,8.00,RU,1603000423
...,...,...,...,...,...,...,...,...,...,...
917,917,Ballina,-28.87,153.57,305.15,57,50,7.70,AU,1603000560
918,918,Qaanaaq,77.48,-69.36,275.26,83,99,4.59,GL,1603000471
919,919,Arraial do Cabo,-22.97,-42.03,294.21,80,53,5.76,BR,1603000509
920,920,Rikitea,-23.12,-134.97,296.97,78,100,9.03,PF,1603000432


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
humidity = cities["Humidity"].astype(float)
locations = cities[["Lat","Lng"]]
max_temp = humidity.max()
locations

,Lat,Lng
0,47.94,-97.43
1,-46.19,168.86
2,-54.80,-68.30
3,-54.80,-68.30
4,56.80,59.93
...,...,...
917,-28.87,153.57
918,77.48,-69.36
919,-22.97,-42.03
920,-23.12,-134.97


In [78]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_temp,point_radius=3)
fig.add_layer(heat_layer)
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid red',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
gmaps.figure(layout = figure_layout)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [82]:
new_cities = cities.loc[(cities["Max Temp"]>70) & (cities["Max Temp"]>80) & (cities["Wind Speed"]<10) & (cities["Cloudiness"] == 0),:]
new_cities = new_cities.dropna(how = "any")
new_cities = new_cities.reset_index()
new_cities



,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,14,14,Bowen,-20.02,148.23,302.15,39,0,5.70,AU,1603000425
1,36,36,Sergeevka,53.88,67.42,276.99,70,0,5.62,KZ,1603000430
2,66,66,Busselton,-33.65,115.33,300.37,29,0,1.90,AU,1603000437
3,67,67,Neuquén,-38.95,-68.06,289.15,48,0,4.60,AR,1603000437
4,81,81,Valparaíso,-33.04,-71.63,284.82,72,0,5.14,CL,1603000440
...,...,...,...,...,...,...,...,...,...,...,...
94,901,901,Jiayuguan City,39.82,98.30,287.28,29,0,6.10,CN,1603000615
95,902,902,Whitecourt,54.15,-115.69,264.53,85,0,1.29,CA,1603000615
96,907,907,Buraidah,26.33,43.98,291.15,34,0,0.64,SA,1603000557
97,909,909,Erenhot,43.65,111.98,285.15,38,0,6.12,CN,1603000617


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [83]:
hotel_name = []
for i in range(len(new_cities)):
    lat = new_cities["Lat"][i]
    lng = new_cities["Lng"][i]
    location = f'{lat},{lng}',
    params = {
        "location":location,
        "radius":5000,
        "types": "hotel",
        "key":g_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url,params = params)
    json = requested.json()
    try:
        hotel_name.append(json['results'][0]['name'])
    except(KeyError, IndexError):
        hotel_name.append(np.nan)


In [84]:
hotel_df = new_cities
hotel_df["Hotel Name"] = hotel_name
hotel_df = hotel_df.dropna(how ='any')
hotel_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,14,14,Bowen,-20.02,148.23,302.15,39,0,5.70,AU,1603000425,Bowen
1,36,36,Sergeevka,53.88,67.42,276.99,70,0,5.62,KZ,1603000430,Sergeevka
2,66,66,Busselton,-33.65,115.33,300.37,29,0,1.90,AU,1603000437,Busselton
3,67,67,Neuquén,-38.95,-68.06,289.15,48,0,4.60,AR,1603000437,Neuquen
4,81,81,Valparaíso,-33.04,-71.63,284.82,72,0,5.14,CL,1603000440,Valparaíso
...,...,...,...,...,...,...,...,...,...,...,...,...
94,901,901,Jiayuguan City,39.82,98.30,287.28,29,0,6.10,CN,1603000615,Jiayuguan City
95,902,902,Whitecourt,54.15,-115.69,264.53,85,0,1.29,CA,1603000615,Whitecourt
96,907,907,Buraidah,26.33,43.98,291.15,34,0,0.64,SA,1603000557,Buraydah
97,909,909,Erenhot,43.65,111.98,285.15,38,0,6.12,CN,1603000617,Xilingol League


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))